In [40]:
# Segun el grafo. Generar el archivo CSV, obtener el mejor recorrido usando algoritmos genéticos sin el uso de DEAP.
import random
import pandas as pd
import numpy

population = []  # lista que contiene rutas
population_size = 10  # Tamaño
mutate_prob = 0.1 
n_generations = 100 # num de generaciones o ruas
routes_length = [0]*population_size
fitness = [0]*population_size
best_path = 1000
best = []


In [41]:
cities = [0, 1, 2, 3, 4]; cities14 = [1, 2, 3, 4]
df= pd.read_csv(r'C:\Users\ivanj\Desktop\INF354_2do\Ejercicio 3\grafo.csv', skiprows=0, low_memory=False)
distances = numpy.array(df.values)
distances

array([[ 0,  7,  9,  8, 20],
       [ 7,  0, 10,  4, 11],
       [ 9, 10,  0, 15,  5],
       [ 8,  4, 15,  0, 17],
       [20, 11,  5, 17,  0]], dtype=int64)

In [42]:
# Calcular distancia
def calc_distance(city1, city2):
    return distances[city1][city2]  # ord('A')=65

# Crear Ruta
def create_route():
    shuffled = [0]
    shuffled += random.sample(cities14, len(cities14))
    return shuffled

# Calcular la longitud de una ruta
def calc_route_length():
    for i in range(population_size):
        route_l = 0
        for j in range(0, len(cities)):
            route_l = route_l + calc_distance(population[i][j - 1], population[i][j])
        routes_length[i] = route_l
        fitness[i] = 1 / routes_length[i]

# Crear población
def create_population():
    for i in range(population_size):
        population.append(create_route())    

# Mutación
def swap_mutation(ind):
    picks = random.sample(range(len(cities)), 2)
    temp = population[ind][picks[0]]
    population[ind][picks[0]] = population[ind][picks[1]]
    population[ind][picks[1]] = temp
    
# Cruce
def partially_matched_crossover(ind1, ind2):
    size = len(cities)
    p1, p2 = [0] * size, [0] * size

    # Inicializar la posición de cada índice en los individuos
    for k in range(size):
        p1[ind1[k]] = k
        p2[ind2[k]] = k
    # Choose crossover points
    cxpoint1 = random.randint(0, size)
    cxpoint2 = random.randint(0, size - 1)
    if cxpoint2 >= cxpoint1:
        cxpoint2 += 1
    else:  # Intercambiar los dos puntos cx
        cxpoint1, cxpoint2 = cxpoint2, cxpoint1

# Aplicar cruce entre puntos cx
    for k in range(cxpoint1, cxpoint2):
    # Realizar un seguimiento de los valores seleccionados
        temp1 = ind1[k]
        temp2 = ind2[k]
    # Intercambiar el valor coincidente
        ind1[k], ind1[p1[temp2]] = temp2, temp1
        ind2[k], ind2[p2[temp1]] = temp1, temp2
    # Contabilidad de posiciones
        p1[temp1], p1[temp2] = p1[temp2], p1[temp1]
        p2[temp1], p2[temp2] = p2[temp2], p2[temp1]

    return ind1, ind2



In [43]:
# selección de la mejor ruta
def roulette_wheel_selection():
    s = 0
    partial_s = 0
    ind = 0
    for m in range(population_size):
        s = s + fitness[m]
    rand = random.uniform(0, s)
    for m in range(population_size):
        if partial_s < rand:
            partial_s = partial_s + fitness[m]
            ind = ind + 1
    if ind == population_size:  # prevenir fuera del limite de la lista
        ind = population_size - 1
    return ind

# la mejor ruta
def find_fittest():
    key = 1000
    fittest = 0
    for i in range(population_size):
        if routes_length[i] < key:
            key = routes_length[i]
            fittest = i
    return fittest

In [44]:
create_population()
calc_route_length()
best = [0,0,0,0,0]
rutas = []
recorrido = []
x = []
for j in range(n_generations):
    for i in range(0, population_size, 2):
       
        parent1 = roulette_wheel_selection()
        parent2 = roulette_wheel_selection()
     
        while True:
            if parent1 == parent2:
                parent2 = roulette_wheel_selection()
            else:
                break
       
        population[i], population[i + 1] = partially_matched_crossover(population[parent1], population[parent2])
       
        calc_route_length()

 
    for i in range(population_size):
        rand = random.uniform(0, 1)
        if rand < mutate_prob:
            swap_mutation(i)

    calc_route_length()

    if routes_length[find_fittest()] < best_path:
        index = find_fittest()
        best_path = routes_length[index]
    x = population[find_fittest()]
    
    print("Ruta ", j+1, ": ", x,"Recorrido: ", routes_length[find_fittest()])
    best+=population[find_fittest()]
    
    rutas.append(routes_length[find_fittest()])
   
    if routes_length[find_fittest()] == best_path:
        gr = j
    
    
    recorrido.append(x)
    
ind=5*(gr+1)
print("\nMejor ruta: [", best[ind], best[ind+1], best[ind+2], best[ind+3], best[ind+4], "]\nDistancia final:",best_path)

 

Ruta  1 :  [0, 3, 4, 2, 1] Recorrido:  47
Ruta  2 :  [0, 1, 3, 4, 2] Recorrido:  42
Ruta  3 :  [0, 2, 3, 1, 4] Recorrido:  59
Ruta  4 :  [4, 0, 1, 3, 2] Recorrido:  51
Ruta  5 :  [4, 0, 1, 3, 2] Recorrido:  51
Ruta  6 :  [0, 1, 4, 3, 2] Recorrido:  59
Ruta  7 :  [2, 3, 4, 1, 0] Recorrido:  59
Ruta  8 :  [2, 3, 4, 1, 0] Recorrido:  59
Ruta  9 :  [2, 3, 4, 1, 0] Recorrido:  59
Ruta  10 :  [2, 3, 4, 1, 0] Recorrido:  59
Ruta  11 :  [2, 0, 4, 3, 1] Recorrido:  60
Ruta  12 :  [3, 1, 4, 2, 0] Recorrido:  37
Ruta  13 :  [1, 3, 4, 2, 0] Recorrido:  42
Ruta  14 :  [1, 3, 4, 2, 0] Recorrido:  42
Ruta  15 :  [4, 3, 1, 2, 0] Recorrido:  60
Ruta  16 :  [4, 2, 1, 3, 0] Recorrido:  47
Ruta  17 :  [2, 4, 1, 3, 0] Recorrido:  37
Ruta  18 :  [2, 3, 4, 0, 1] Recorrido:  69
Ruta  19 :  [2, 3, 4, 0, 1] Recorrido:  69
Ruta  20 :  [2, 3, 4, 0, 1] Recorrido:  69
Ruta  21 :  [2, 3, 4, 0, 1] Recorrido:  69
Ruta  22 :  [3, 2, 4, 0, 1] Recorrido:  51
Ruta  23 :  [3, 2, 4, 0, 1] Recorrido:  51
Ruta  24 :  [3, 2, 4

In [45]:

df_rutas = pd.DataFrame(dict_rutas)
df_rutas.to_csv('rutas.csv', index=True)